In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#EFFICIENTNETB0 MODEL
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define dataset path
DATASET_PATH = "/kaggle/input/glaucomadataset-healthy-and-infected-images/datasets combined/"

# Image Augmentation to increase dataset size
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Load dataset
train_data = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(224, 224),
    batch_size=8,  # Smaller batch size for small datasets
    class_mode='binary',
    subset='training'
)

val_data = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(224, 224),
    batch_size=8,
    class_mode='binary',
    subset='validation'
)

# Load EfficientNetB0 Pre-trained Model
base_model = EfficientNetB0(
    weights="/kaggle/input/efficientnetb0-notop-h5/efficientnetb0_notop.h5",
    include_top=False,
    input_shape=(224, 224, 3)
)

# Add Custom Layers on top
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.4)(x)  # Reduce overfitting
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)  # More dropout
output = Dense(1, activation="sigmoid")(x)  # Binary classification

model = Model(inputs=base_model.input, outputs=output)

# Compile the Model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# Callbacks
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6)

# Train the Model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=30,
    callbacks=[early_stopping, reduce_lr]
)

# Save the model
model.save("glaucoma_model.keras")

print("Training completed and model saved!")

# Compute the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
# Make predictions on the validation set
predicted_classes_efficientnet = model.predict(X_test)  # Replace cnn_model with your actual CNN model variable
predicted_classes_efficientnet = np.argmax(predicted_classes_efficientnet, axis=1)  # In case of multi-class classification, use argmax

efficientnet_accuracy = accuracy_score(y_test, predicted_classes_cnn)  # Now using the defined predicted_classes_cnn
efficientnet_report = classification_report(y_test, predicted_classes_efficientnet, output_dict=True)
efficientnet_precision = efficientnet_report['accuracy']
efficientnet_recall = efficientnet_report['macro avg']['recall']
efficientnet_f1 = efficientnet_report['macro avg']['f1-score']

# Add the results to the list
results.append({
    "Model": "efficientnet",
    "Accuracy": efficient_accuracy,
    "Precision": efficient_precision,
    "Recall": efficient_recall,
    "F1-Score": efficient_f1
})

sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy')

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Model Loss')

plt.show()

In [ ]:
#MOBILENETV2 MODEL
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models, optimizers, callbacks

# Define paths
data_dir = "/kaggle/input/glaucomadataset-healthy-and-infected-images/datasets combined"

# Image Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Load Training and Validation Data
train_data = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_data = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Load MobileNetV2 Pre-trained Model
base_model = MobileNetV2(weights="/kaggle/input/mobilenet/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5",
                                 include_top=False, input_shape=(224, 224, 3))


# Build Model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')  # Binary classification (Healthy vs Glaucoma)
])

# Compile Model
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Callbacks (Early Stopping & Reduce LR on Plateau)

reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)

# Train Model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=17,
    callbacks=[reduce_lr]
)

# Save Model
model.save("mobilenetv2_glaucoma_model.keras")

print("Training completed and model saved!")

# Compute the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
# Make predictions on the validation set
predictions = model.predict(val_data)  # Using validation set for predictions

# Convert probabilities to class labels
predicted_classes = (predictions > 0.5).astype("int32")  # Threshold at 0.5 for binary classification

# Assuming val_data has the true labels as well
true_labels = val_data.classes

cm = confusion_matrix(true_labels, predicted_classes)
print("Confusion Matrix:\n", cm)
report = classification_report(true_labels, predicted_classes, target_names=['Healthy', 'Glaucoma'])
print("Classification Report:\n", report)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy')

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Model Loss')

plt.show()

In [ ]:
#DENSENET121
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from PIL import Image

# Define paths
dataset_path = "/kaggle/input/glaucomadataset-healthy-and-infected-images/datasets combined"
batch_size = 16  # Adjust based on Kaggle memory limits
img_size = (224, 224)  # Resize images for DenseNet

# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Splitting into training and validation
)

# Load training images
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

# Load validation images
val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Define the correct path to your local weights file
weights_path = "/kaggle/input/dense121/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"

# Load DenseNet121 model without the top layers (for feature extraction)
base_model = tf.keras.applications.DenseNet121(
    include_top=False,  # Remove the fully connected layers
    input_shape=(224, 224, 3),  # Adjust the input shape if needed
    weights=None  # Do not fetch from the internet
)

# Load the weights from the local file
base_model.load_weights(weights_path)

# Set the base model to not trainable (freeze it)
base_model.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)  # Pooling layer to flatten the output
x = tf.keras.layers.Dense(1024, activation='relu')(x)  # Fully connected layer
x = tf.keras.layers.Dropout(0.5)(x)  # Dropout layer to reduce overfitting
predictions = tf.keras.layers.Dense(1, activation='sigmoid')(x)  # 1 class: healthy or glaucoma (binary classification)

# Final model
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

# Compile the model with binary_crossentropy loss function
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',  # Use binary_crossentropy for binary classification
              metrics=['accuracy'])

# Train the model using the generators
history = model.fit(
    train_generator,
    epochs=15,  # Set the number of epochs for training
    validation_data=val_generator,
    verbose=1
)



In [ ]:
#ADJUSTING DENSENET121 THROUGH
# 1.DATA AUGUMENTATION AND L2 REGULARIZATION
# 2.DROPOUT
# 3.LEARNING RATE REDUCTION AND EARLY STOPPING


import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import regularizers
from PIL import Image


# Define paths
dataset_path = "/kaggle/input/glaucomadataset-healthy-and-infected-images/datasets combined"
batch_size = 16  # Adjust based on Kaggle memory limits
img_size = (224, 224)  # Resize images for DenseNet

# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Splitting into training and validation
)

# Load training images
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

# Load validation images
val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Define the correct path to your local weights file
weights_path = "/kaggle/input/dense121/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"

# Load DenseNet121 model without the top layers (for feature extraction)
base_model = tf.keras.applications.DenseNet121(
    include_top=False,  # Remove the fully connected layers
    input_shape=(224, 224, 3),  # Adjust the input shape if needed
    weights=None  # Do not fetch from the internet
)

# Load the weights from the local file
base_model.load_weights(weights_path)

# Set the base model to not trainable (freeze it)
base_model.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)  # Pooling layer to flatten the output
x = tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)  # L2 Regularization
x = tf.keras.layers.Dropout(0.5)(x)  # Dropout layer to reduce overfitting
predictions = tf.keras.layers.Dense(1, activation='sigmoid')(x)  # 1 class: healthy or glaucoma (binary classification)

# Final model
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

# Compile the model with binary_crossentropy loss function
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',  # Use binary_crossentropy for binary classification
              metrics=['accuracy'])

# Early stopping and reduce learning rate on plateau
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

# Train the model using the generators
history = model.fit(
    train_generator,
    epochs=13,  # Set the number of epochs for training
    validation_data=val_generator,
    verbose=1,
    callbacks=[early_stopping, reduce_lr]  # Use EarlyStopping and Learning Rate Reduction
)
# Make predictions on the validation set
predictions = model.predict(val_generator)  # Using validation set for predictions

# Convert probabilities to class labels (threshold at 0.5 for binary classification)
predicted_classes = (predictions > 0.5).astype("int32")

# Compute the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming val_generator has the true labels as well
true_labels = val_generator.classes

cm = confusion_matrix(true_labels, predicted_classes)
print("Confusion Matrix:\n", cm)

# Classification Report (Precision, Recall, F1-score)
report = classification_report(true_labels, predicted_classes, target_names=['Healthy', 'Glaucoma'])
print("Classification Report:\n", report)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

# Plotting accuracy and loss
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy')

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Model Loss')

plt.show()


In [ ]:
#CNN MODEL
import os
import numpy as np
import cv2
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

# Set image size
IMG_SIZE = 224

# Define paths
healthy_path = "/kaggle/input/glaucomadataset-healthy-and-infected-images/datasets combined/healthy"
glaucoma_path = "/kaggle/input/glaucomadataset-healthy-and-infected-images/datasets combined/glaucoma_project"

# Initialize data lists
X = []
y = []

# Load healthy images (Label = 0)
for img_name in os.listdir(healthy_path):
    img_path = os.path.join(healthy_path, img_name)
    img = cv2.imread(img_path)
    if img is not None:  # Check if image is valid
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) / 255.0  # Normalize
        X.append(img)
        y.append(0)  # Label for healthy

# Load glaucoma-infected images (Label = 1)
for img_name in os.listdir(glaucoma_path):
    img_path = os.path.join(glaucoma_path, img_name)
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) / 255.0
        X.append(img)
        y.append(1)  # Label for glaucoma

# Convert to NumPy arrays
X = np.array(X)
y = np.array(y)

# Split into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Data Augmentation to artificially increase dataset size
datagen = ImageDataGenerator(
    rescale=1./255,           # Normalize pixel values
    rotation_range=30,        # Random rotation
    width_shift_range=0.2,    # Horizontal shift
    height_shift_range=0.2,   # Vertical shift
    shear_range=0.2,          # Shearing
    zoom_range=0.2,           # Zooming
    horizontal_flip=True,     # Flipping images
    fill_mode='nearest'       # Fill in missing pixels
)

# CNN model
model = Sequential([
    Input(shape=(IMG_SIZE, IMG_SIZE, 3)),  # Define input shape separately

    # First Convolutional Layer
    Conv2D(32, (3,3), activation="relu"),
    MaxPooling2D(pool_size=(2,2)),

    # Second Convolutional Layer
    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D(pool_size=(2,2)),

    # Third Convolutional Layer
    Conv2D(128, (3,3), activation="relu"),
    MaxPooling2D(pool_size=(2,2)),

    # Flatten and Fully Connected Layers
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),  # Prevent overfitting
    Dense(1, activation="sigmoid")  # Output layer for binary classification
])

# Choose optimizer (You can switch between 'adam' and 'rmsprop')
optimizer = Adam(learning_rate=0.001)  # You can change to RMSprop(learning_rate=0.001)

# Compile the model
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

# ReduceLROnPlateau - Automatically adjusts learning rate if model stops improving
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=0.00001)

# Train the model with data augmentation
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=4),  # Apply data augmentation
    validation_data=(X_test, y_test),
    epochs=15,  # Increased from default (Try adjusting)
    callbacks=[lr_reduction],  # Reduce learning rate when necessary
    verbose=1
)

# CNN Model - ensure this part gets defined after training the CNN model
predicted_classes_cnn = model.predict(X_test)  # Replace cnn_model with your actual CNN model variable
predicted_classes_cnn = np.argmax(predicted_classes_cnn, axis=1)  # In case of multi-class classification, use argmax

cnn_accuracy = accuracy_score(y_test, predicted_classes_cnn)  # Now using the defined predicted_classes_cnn
cnn_report = classification_report(y_test, predicted_classes_cnn, output_dict=True)
cnn_precision = cnn_report['accuracy']
cnn_recall = cnn_report['macro avg']['recall']
cnn_f1 = cnn_report['macro avg']['f1-score']

# Add the results to the list
results.append({
    "Model": "CNN",
    "Accuracy": cnn_accuracy,
    "Precision": cnn_precision,
    "Recall": cnn_recall,
    "F1-Score": cnn_f1
})


sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

# Plot training accuracy and loss
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy')

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Model Loss')

plt.show()

In [ ]:
#LOGISTIC REGRESSION MODEL
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Define paths
healthy_path = "/kaggle/input/glaucomadataset-healthy-and-infected-images/datasets combined/healthy"
glaucoma_path = "/kaggle/input/glaucomadataset-healthy-and-infected-images/datasets combined/glaucoma_project"

# Image settings
IMG_SIZE = 224  # Resize images to 224x224

# Prepare data storage
X = []  # Image data
y = []  # Labels (0 = Healthy, 1 = Glaucoma)

# Load healthy images
for img_name in os.listdir(healthy_path):
    img = cv2.imread(os.path.join(healthy_path, img_name))
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) / 255.0  # Normalize
    X.append(img)
    y.append(0)  # Label for healthy

# Load glaucoma-infected images
for img_name in os.listdir(glaucoma_path):
    img = cv2.imread(os.path.join(glaucoma_path, img_name))
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) / 255.0
    X.append(img)
    y.append(1)  # Label for glaucoma

# Convert to NumPy arrays
X = np.array(X)
y = np.array(y)

# Split into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Total images: {len(X)}, Training: {len(X_train)}, Testing: {len(X_test)}")

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Flatten images for logistic regression (convert 2D to 1D)
X_train_flat = X_train.reshape(len(X_train), -1)
X_test_flat = X_test.reshape(len(X_test), -1)

# Train Logistic Regression model
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train_flat, y_train)

# Predict and evaluate
y_pred = log_model.predict(X_test_flat)
accuracy = accuracy_score(y_test, y_pred)
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Logistic Regression
log_cm = confusion_matrix(y_test, y_pred)
print("Logistic Regression Confusion Matrix:\n", log_cm)
print("Logistic Regression Classification Report:\n", classification_report(y_test, y_pred))


print(f"Baseline Model (Logistic Regression) Accuracy: {accuracy:.2f}")
